<a href="https://colab.research.google.com/github/Tavera523/TareasCursos20B/blob/main/Modelo_Posicional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import math
import time
from collections import defaultdict
import csv
from celluloid import Camera
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg') # Use 'Agg' backend for non-interactive environments

area_km2 = 80.0
area_m2 = area_km2 * 1e6
side_m = math.sqrt(area_m2)

population = 800000
infection_radius = 30.0
base_infectious_prob = 1
decay_scale = infection_radius / 1.5
infectious_period = 30
move_scale = 1.0
initial_infected = 50

default_timesteps = 200

COLOR_SUSCEPTIBLE = 'yellow'
COLOR_INFECTED = 'blue'
COLOR_RECOVERED = 'green'

class SpatialHash:
    def __init__(self, side_m, cell_size):
        self.side = side_m
        self.cell = cell_size
        self.n_cells = int(math.ceil(side_m / cell_size))
        self.cells = defaultdict(list)
    def _cell_index(self, x, y):
        ix = int(x // self.cell)
        iy = int(y // self.cell)
        return ix, iy
    def build(self, positions):
        self.cells.clear()
        for i, (x,y) in enumerate(positions):
            ix, iy = self._cell_index(x, y)
            self.cells[(ix,iy)].append(i)
    def neighbors_within(self, positions, index, radius):
        x,y = positions[index]
        r = radius
        min_ix = int(max(0, (x - r) // self.cell))
        max_ix = int(min(self.n_cells-1, (x + r) // self.cell))
        min_iy = int(max(0, (y - r) // self.cell))
        max_iy = int(min(self.n_cells-1, (y + r) // self.cell))
        result = []
        for ix in range(min_ix, max_ix+1):
            for iy in range(min_iy, max_iy+1):
                for j in self.cells.get((ix,iy), []):
                    if j == index: continue
                    dx = positions[j,0] - x
                    dy = positions[j,1] - y
                    if dx*dx + dy*dy <= r*r:
                        result.append((j, math.hypot(dx,dy)))
        return result
def run_simulation(population=population, timesteps=default_timesteps, seed=123, animate=False, save_stats_csv=None):
    np.random.seed(seed)
    N = population
    side = side_m
    positions = np.random.rand(N,2) * side
    state = np.zeros(N, dtype=np.int8)  # 0=S, 1=I, 2=R
    days_infected = np.zeros(N, dtype=np.int32)
    init_idx = np.random.choice(N, size=min(initial_infected,N), replace=False)
    state[init_idx] = 1
    days_infected[init_idx] = 0
    grid = SpatialHash(side, infection_radius)

    stats = {'S':[], 'I':[], 'R':[]}
    t0 = time.time()

    if animate:
        plt.close('all') # Ensure a clean slate for the animation figure
        fig, ax = plt.subplots(figsize=(6,6))
        camera = Camera(fig)

    for t in range(timesteps):
        movements = np.random.normal(scale=move_scale, size=(N,2))
        positions += movements
        positions = np.clip(positions, 0, side - 1e-6)
        grid.build(positions)
        newly_infected = []
        infected_indices = np.where(state==1)[0]
        for i in infected_indices:
            neighs = grid.neighbors_within(positions, i, infection_radius)
            for j, dist in neighs:
                if state[j] != 0: continue
                p = base_infectious_prob * math.exp(-dist / decay_scale)
                if np.random.rand() < p:
                    newly_infected.append(j)
        for j in newly_infected:
            state[j] = 1
            days_infected[j] = 0
        if infected_indices.size > 0:
            days_infected[infected_indices] += 1
            recovered = infected_indices[days_infected[infected_indices] >= infectious_period]
            if recovered.size > 0:
                state[recovered] = 2

        S = int((state==0).sum())
        I = int((state==1).sum())
        R = int((state==2).sum())
        stats['S'].append(S); stats['I'].append(I); stats['R'].append(R)
        if t % max(1, timesteps//10) == 0:
            print(f"t={t} S={S} I={I} R={R}")

        if animate:
            ax.clear()
            ax.set_title(f"Propagación - Día {t}")
            ax.set_xlim(0, side)
            ax.set_ylim(0, side)

            colors = np.where(state==0, COLOR_SUSCEPTIBLE,
                      np.where(state==1, COLOR_INFECTED, COLOR_RECOVERED))

            ax.scatter(positions[:,0], positions[:,1], s=1, c=colors)
            fig.canvas.draw() # Explicitly draw the canvas before snapping
            camera.snap()

    runtime = time.time() - t0
    print(f"Simulation finished in {runtime:.2f} s")

    if save_stats_csv:
        with open(save_stats_csv, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['t','S','I','R'])
            for t in range(timesteps):
                writer.writerow([t, stats['S'][t], stats['I'][t], stats['R'][t]])
        print(f"Stats saved to {save_stats_csv}")

    if animate:
        animation = camera.animate(interval=80)
        animation.save("propagacion.gif", writer="pillow")
        print("Animación guardada como propagacion.gif")
        plt.close(fig) # Close the animation figure after saving

        plt.figure(figsize=(8,4))
        plt.plot(stats['S'], label='Susceptibles', color='yellow')
        plt.plot(stats['I'], label='Infectados', color='blue')
        plt.plot(stats['R'], label='Recuperados', color='green')
        plt.xlabel("Tiempo (días)")
        plt.ylabel("Número de individuos")
        plt.title("Curvas SIR de la simulación")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig("curva_SIR.png")
        plt.show()
        print("Gráfica SIR guardada como curva_SIR.png")
        plt.close(plt.gcf()) # Close the SIR curve figure

    return stats

if __name__ == '__main__':
    example_stats = run_simulation(population=80000, timesteps=100, seed=42, animate=True, save_stats_csv='sir_stats_example.csv')
    print('Example run complete.')

t=0 S=79894 I=106 R=0
t=10 S=79467 I=533 R=0
t=20 S=79133 I=867 R=0
t=30 S=78784 I=1110 R=106
t=40 S=78505 I=962 R=533
t=50 S=78232 I=901 R=867
t=60 S=78036 I=748 R=1216
t=70 S=77877 I=628 R=1495
t=80 S=77750 I=482 R=1768
t=90 S=77643 I=393 R=1964
Simulation finished in 176.80 s
Stats saved to sir_stats_example.csv


IndexError: list index out of range

In [ ]:
import pandas as pd

# Load the CSV file into a pandas DataFrame
df_sir_stats = pd.read_csv('sir_stats_example.csv')

# Display the first 5 rows of the DataFrame
print("First 5 rows of sir_stats_example.csv:")
display(df_sir_stats.head())

First 5 rows of sir_stats_example.csv:


,t,S,I,R
0,0,79894,106,0
1,1,79838,162,0
2,2,79787,213,0
3,3,79736,264,0
4,4,79692,308,0


from matplotlib import pyplot as plt
_df_12['t'].plot(kind='hist', bins=20, title='t')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_13['S'].plot(kind='hist', bins=20, title='S')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_14['I'].plot(kind='hist', bins=20, title='I')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['I']
  ys = series['S']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_19.sort_values('I', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('I')
_ = plt.ylabel('S')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['I']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'I'}, axis=1)
              .sort_values('I', ascending=True))
  xs = counted['I']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_20.sort_values('I', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('I')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_21['t'].plot(kind='line', figsize=(8, 4), title='t')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_22['S'].plot(kind='line', figsize=(8, 4), title='S')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_23['I'].plot(kind='line', figsize=(8, 4), title='I')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(df_sir_stats['t'], df_sir_stats['S'], label='Susceptible', color='blue')
plt.plot(df_sir_stats['t'], df_sir_stats['I'], label='Infected', color='red')
plt.plot(df_sir_stats['t'], df_sir_stats['R'], label='Recovered', color='green')

plt.xlabel('Time (days)')
plt.ylabel('Number of Individuals')
plt.title('SIR Model Progression')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Display basic information about the DataFrame, including data types and non-null values
print("\nDataFrame Info:")
df_sir_stats.info()


DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   t       100 non-null    int64
 1   S       100 non-null    int64
 2   I       100 non-null    int64
 3   R       100 non-null    int64
dtypes: int64(4)
memory usage: 3.3 KB


In [ ]:
# Display descriptive statistics for the numerical columns
print("\nDescriptive Statistics:")
display(df_sir_stats.describe())


Descriptive Statistics:


,t,S,I,R
count,100.000000,100.00000,100.000000,100.000000
mean,49.500000,78421.00000,689.830000,889.170000
std,29.011492,688.93585,259.164589,766.144355
min,0.000000,77530.00000,106.000000,0.000000
25%,24.750000,77806.50000,456.000000,0.000000
50%,49.500000,78243.00000,690.500000,847.500000
75%,74.250000,78968.75000,931.750000,1618.750000
max,99.000000,79894.00000,1144.000000,2106.000000


from matplotlib import pyplot as plt
_df_24['t'].plot(kind='hist', bins=20, title='t')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_25['S'].plot(kind='hist', bins=20, title='S')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_26['I'].plot(kind='hist', bins=20, title='I')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['t']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 't'}, axis=1)
              .sort_values('t', ascending=True))
  xs = counted['t']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_34.sort_values('t', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('t')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_35['t'].plot(kind='line', figsize=(8, 4), title='t')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_36['S'].plot(kind='line', figsize=(8, 4), title='S')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_37['I'].plot(kind='line', figsize=(8, 4), title='I')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_38['R'].plot(kind='line', figsize=(8, 4), title='R')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
pip install celluloid
